In [1]:
'''
计算情感得分，基于boson词典。
'''
import jieba
import pandas as pd

sentences = pd.read_csv('../data/sentence_sentiment.csv', sep='\t')
sentences_zero = sentences[sentences['sentiment'] == 0]
sentences_not_zero = sentences[sentences['sentiment'] != 0]
sentences_zero.shape, sentences_not_zero.shape

((880239, 2), (424545, 2))

In [3]:
# 读取文件，文件读取函数
def read_file(filename):
    # with open(filename, 'rb')as f:
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()
        # 返回list类型数据
        text = text.split('\n')
    return text


# 读取所需文件
# 读取停用词表
stop_words = read_file(r"../data/baidu_stopwords.txt")
# 去掉停用词中的情感词 Boson
# 情感词与停用词有重合导致一些文本分数为0
stop_df = pd.DataFrame(stop_words)
boson_df = pd.read_excel('../data/8_BosonNLP_sentiment_lexicon.xlsx')
stop_df.columns = ['word']
duplicated = pd.merge(stop_df, boson_df, on='word')['word'].tolist()
stop_words = list(filter(lambda x: x not in duplicated, stop_words))

most = read_file("../data/lexicons/most.txt")
very = read_file("../data/lexicons/very.txt")
more = read_file("../data/lexicons/more.txt")
ish = read_file("../data/lexicons/ish.txt")
insufficiently = read_file("../data/lexicons/insufficiently.txt")
inverse = read_file("../data/lexicons/inverse.txt")


# 读取情感词及分数
def get_senti_word(polar):
    """
    读取情感词，Boson或Kansei
    :param polar: pos or neg
    :return: {sentiment word: score}
    """

    if polar == 'pos':
        pos_senti = boson_df[boson_df['sentiment'] > 0]
        senti_dict = pos_senti.set_index(keys='word')['sentiment'].to_dict()
        return senti_dict
    elif polar == 'neg':
        neg_senti = boson_df[boson_df['sentiment'] < 0]
        senti_dict = neg_senti.set_index(keys='word')['sentiment'].to_dict()
        return senti_dict


# 去停用词函数
def del_stopwords(words):
    # 去除停用词后的句子
    new_words = []
    for word in words:
        if word not in stop_words:
            new_words.append(word)
    return new_words


# 获取六种权值的词，根据要求返回list，这个函数是为了配合Django的views下的函数使用
def weighted_value(request):
    result_dict = []
    if request == "most":
        result_dict = most
    elif request == "very":
        result_dict = very
    elif request == "more":
        result_dict = more
    elif request == "ish":
        result_dict = ish
    elif request == "insufficiently":
        result_dict = insufficiently
    elif request == "inverse":
        result_dict = inverse
    elif request == 'pos_dict':
        result_dict = get_senti_word(polar='pos')
    elif request == 'neg_dict':
        result_dict = get_senti_word(polar='neg')
    else:
        pass
    return result_dict


print("reading sentiment dict .......")
# 读取情感词典
pos_dict = weighted_value('pos_dict')
neg_dict = weighted_value('neg_dict')
# 读取程度副词词典
# 权值为2
most_dict = weighted_value('most')
# 权值为1.75
very_dict = weighted_value('very')
# 权值为1.50
more_dict = weighted_value('more')
# 权值为1.25
ish_dict = weighted_value('ish')
# 权值为0.25
insufficient_dict = weighted_value('insufficiently')
# 权值为-1
inverse_dict = weighted_value('inverse')


# 程度副词处理，对不同的程度副词给予不同的权重
def match_adverb(word, sentiment_value):
    # 最高级权重为
    if word in most_dict:
        sentiment_value *= 8
    # 比较级权重
    elif word in very_dict:
        sentiment_value *= 6
    # 比较级权重
    elif word in more_dict:
        sentiment_value *= 4
    # 轻微程度词权重
    elif word in ish_dict:
        sentiment_value *= 2
    # 相对程度词权重
    elif word in insufficient_dict:
        sentiment_value *= 0.5
    # 否定词权重
    elif word in inverse_dict:
        sentiment_value *= -1
    else:
        sentiment_value *= 1
    return sentiment_value


# 对每一条微博打分
def single_sentiment_score(sent):
    if pd.isna(sent):
        return -2
    # 分词
    words = list(jieba.cut(sent))
    seg_words = del_stopwords(words)
    # i，s 记录情感词和程度词出现的位置
    i = 0  # 记录扫描到的词位置
    s = 0  # 记录情感词的位置
    pos_score = []  # 记录正向情感分数
    neg_score = []  # 记录负向情感分数

    # 逐个查找情感词
    for word in seg_words:
        # 如果为积极词汇
        if word in pos_dict.keys():
            pos_word_score = pos_dict.get(word)
            # 在情感词前面寻找程度副词
            for w in seg_words[s:i]:
                pos_word_score = match_adverb(w, pos_word_score)
            pos_score.append(pos_word_score)
            s = i + 1  # 记录情感词位置
            # 如果是消极情感词
        elif word in neg_dict.keys():
            neg_word_score = neg_dict.get(word)
            for w in seg_words[s:i]:
                neg_word_score = match_adverb(w, neg_word_score)
            neg_score.append(neg_word_score)
            s = i + 1
        i += 1  # 定位情感词的位置
    # 计算情感值
    sentiment_score = sum(pos_score) + sum(neg_score)

    return sentiment_score

reading sentiment dict .......


In [4]:
sentences_zero['sentiment'] = sentences_zero['sentence'].apply(single_sentiment_score)
sentences_zero[sentences_zero['sentiment'] == 0].shape

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\62774\AppData\Local\Temp\jieba.cache
Loading model cost 0.662 seconds.
Prefix dict has been built successfully.
C:\Users\62774\AppData\Local\Temp\ipykernel_10828\358592886.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentences_zero['sentiment'] = sentences_zero['sentence'].apply(single_sentiment_score)


(27902, 2)

In [5]:
boson_zero = sentences_zero[sentences_zero['sentiment'] == 0]
boson_not_zero = sentences_zero[sentences_zero['sentiment'] != 0]
boson_zero.shape, boson_not_zero.shape

((27902, 2), (852337, 2))

In [6]:
sentences_not_zero['dict'] = 'kansei'
boson_not_zero['dict'] = 'boson'
sentence_score = pd.concat([sentences_not_zero, boson_not_zero])
sentence_score

C:\Users\62774\AppData\Local\Temp\ipykernel_10828\2283969600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentences_not_zero['dict'] = 'kansei'
C:\Users\62774\AppData\Local\Temp\ipykernel_10828\2283969600.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boson_not_zero['dict'] = 'boson'


,sentence,sentiment,dict
0,很不错,3.273251,kansei
2,很好很方便,53.832072,kansei
3,交通很方便,1.495335,kansei
4,房间很干净,4.909877,kansei
5,虽然房间面积普遍都不大,-0.220582,kansei
...,...,...,...
1304777,适合孩子,0.233304,boson
1304778,就是房间漏水,-0.264336,boson
1304779,没有热水,-0.113943,boson
1304780,巴适的板,0.450991,boson


In [7]:
sentence_score.to_csv('../data/sentiment_score.csv', sep='\t')
boson_zero.to_csv('../data/sentiment_score_zero.csv', sep='\t')